In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import glob
import re
import os
from src.consts import PATHS, MODEL_SIZES_PER_ARCH_TO_MODEL_ID
from src.types import DATASETS, MODEL_ARCH, DatasetArgs
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from collections import defaultdict
from tqdm.auto import tqdm
import src.plots as plots

import json

from collections import defaultdict
import re
from PIL import Image, ImageDraw, ImageFont



In [17]:
def convert_json_to_df(json_path):
    data = json.load(open(json_path))
    flattened_data = []
    for row in data:
        # Each 'row' contains a single list of lists
        tokens = row[0]
        flattened_row = []
        for token in tokens:
            flattened_row.extend(token)  # Add token_id, token, and prob sequentially
        flattened_data.append(flattened_row)

    # Define column names
    columns = []
    for i in range(1, 6):  # There are 5 tokens per entry
        columns.extend([f"token_id_{i}", f"token_{i}", f"prob_{i}"])

    # Create DataFrame
    df = pd.DataFrame(flattened_data, columns=columns)

    # Display the DataFrame
    return df.loc[:, ~df.columns.str.startswith("token_id")]

In [21]:
base_path = PATHS.OUTPUT_DIR / "state-spaces" / "mamba-1.4B-hf" / "info_flow_test_top_outputs" / "ds=counter_fact" / "ws=9"

In [24]:
for file in base_path.rglob("*.json"):
    print(file.parent.name)
    df = convert_json_to_df(file)
    display(df.head())

block_last_target_last


,token_1,prob_1,token_2,prob_2,token_3,prob_3,token_4,prob_4,token_5,prob_5
0,Microsoft,0.988551,the,0.004111,Windows,0.002642,\n,0.000602,a,0.000526
1,English,0.970354,British,0.023232,called,0.001686,mainly,0.001295,predominantly,0.001087
2,French,0.998235,English,0.001114,the,0.000478,Spanish,0.000065,German,0.000029
3,French,0.999604,not,0.000153,a,0.000081,English,0.000067,the,0.000033
4,Google,0.999688,Google,0.000094,the,0.000088,google,0.000058,a,0.000031


block_first_target_last


,token_1,prob_1,token_2,prob_2,token_3,prob_3,token_4,prob_4,token_5,prob_5
0,Microsoft,0.411915,Dell,0.130631,the,0.128028,Hew,0.041567,a,0.019571
1,English,0.385066,Scottish,0.057332,called,0.051970,Welsh,0.034793,British,0.034365
2,French,0.536126,the,0.110813,English,0.053705,Spanish,0.047490,Occ,0.013906
3,French,0.268134,Arabic,0.089241,not,0.045817,the,0.033795,a,0.027731
4,Google,0.753236,the,0.021991,a,0.016299,Al,0.009493,Facebook,0.007349


block_subject_target_last


,token_1,prob_1,token_2,prob_2,token_3,prob_3,token_4,prob_4,token_5,prob_5
0,Microsoft,0.405084,the,0.133913,Dell,0.124023,Hew,0.043154,a,0.020084
1,English,0.389243,Scottish,0.054253,called,0.053465,British,0.036782,known,0.035049
2,French,0.537603,the,0.110781,English,0.052055,Spanish,0.046137,Occ,0.014009
3,French,0.202496,Arabic,0.082507,not,0.050179,the,0.033112,a,0.026794
4,Google,0.731936,the,0.023378,a,0.017383,Al,0.009931,Facebook,0.007248


block_relation_target_last


,token_1,prob_1,token_2,prob_2,token_3,prob_3,token_4,prob_4,token_5,prob_5
0,Microsoft,0.957148,the,0.016157,Media,0.009880,\n,0.003732,a,0.001932
1,English,0.823017,British,0.138526,called,0.033764,mainly,0.001499,UK,0.000740
2,French,0.996780,the,0.001886,English,0.000597,\n,0.000468,a,0.000065
3,French,0.999639,not,0.000177,a,0.000061,English,0.000043,the,0.000023
4,Google,0.999774,Google,0.000073,the,0.000043,google,0.000034,mark,0.000017
